# Surprise 기반 영화 추천 시스템
### 고객 movie_rating 기준으로 영화를 추천하는 시스템

- 고객 리뷰 데이터셋: surprise.csv
- 영화이름 데이터셋: movie_name


In [1]:
!pip install surprise
!pip install numpy==1.26.4

### 데이터 로드 : surprise.csv

In [2]:
import pandas as pd
ott = pd.read_csv('surprise.csv')
ott.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676088 entries, 0 to 676087
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     676088 non-null  object
 1   movie_name  676088 non-null  object
 2   rating      676088 non-null  int64 
 3   user_name   676088 non-null  object
dtypes: int64(1), object(3)
memory usage: 20.6+ MB


0. user_id: 유저 고유 id
1. movie_name: 영화 제목
2. rating: 별점

- 총 데이터 656815개
- 누락없음

In [3]:
ott.describe()

,rating
count,676088.000000
mean,6.544472
std,2.032788
min,1.000000
25%,5.000000
50%,7.000000
75%,8.000000
max,10.000000


- rating의 평균은 6.5점대이고, 최소는 1, 최대는 10

In [4]:
ott.describe(include='object')

,user_id,movie_name,user_name
count,676088,676088,676088
unique,1082,99051,1082
top,ur5876717,인터스텔라,kosmasp
freq,9943,914,9943


- user : 1,082 명
- movie : 99,051 개

## 사용자별 데이터 분할
각 사용자의 데이터를 훈련 세트와 테스트 세트로 나눕니다.


서프라이즈 범위 지정 및 데이터셋으로 변경

In [5]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ott[['user_id', 'movie_name', 'rating']], reader)

## surpise 모델 학습 및 예측 진행
 - 사용 모델: SVD

- SVD 예측 및 성능 평가


In [6]:
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Split data into trainset and testset
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Build and train the SVD algorithm
algo = SVD()
algo.fit(trainset)

# 테스트셋에 대한 예측 및 성능 평가
pred = algo.test(testset) # 예측
rmse = accuracy.rmse(pred) # 평가

print("RMSE:", rmse)

RMSE: 1.6669
RMSE: 1.6668546989920334


- RMSE가 1.6293으로 평균 제곱근간의 차가 1.6293정도로 발생함

In [ ]:
# 5개 출력
pred[:5]

[Prediction(uid='ur28393208', iid='The Cure in Orange', r_ui=10.0, est=5.751363615479163, details={'was_impossible': False}),
 Prediction(uid='ur59376203', iid='Koyaanisqatsi', r_ui=7.0, est=7.100004484810192, details={'was_impossible': False}),
 Prediction(uid='ur52780913', iid='Jui kuen II', r_ui=8.0, est=7.127924547379752, details={'was_impossible': False}),
 Prediction(uid='ur85577070', iid='The Glass Castle', r_ui=7.0, est=7.628610183246958, details={'was_impossible': False}),
 Prediction(uid='ur49159818', iid='Omoide no Mânî', r_ui=9.0, est=4.948143463829393, details={'was_impossible': False})]

- user id 기입 후 해당 유저가 본적없는 movie_name을 기입

In [ ]:
uid = 'ur100419895' #유저id
iid = 'Bless the Child' #위 유저가 본적없는 영화 id
test = algo.predict(uid, iid)
print(test)

user: ur100419895 item: Bless the Child r_ui = None   est = 5.30   {'was_impossible': False}


- user: user_id | item: movie_name | est: 예측값

### 교차 검증으로 모델 성능 평가 진행

In [ ]:
from surprise.model_selection import cross_validate
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6425  1.6405  1.6351  1.6328  1.6378  1.6378  0.0035  
MAE (testset)     1.2283  1.2262  1.2213  1.2185  1.2225  1.2234  0.0035  
Fit time          11.84   11.92   12.03   11.88   11.64   11.86   0.13    
Test time         1.19    1.52    1.25    1.24    1.25    1.29    0.12    


{'test_rmse': array([1.64246164, 1.64054823, 1.63512356, 1.63282611, 1.63779929]),
 'test_mae': array([1.22828785, 1.2261597 , 1.22132706, 1.21849333, 1.22252953]),
 'fit_time': (11.842564105987549,
  11.917576551437378,
  12.033776044845581,
  11.880022525787354,
  11.640458822250366),
 'test_time': (1.1934888362884521,
  1.5215094089508057,
  1.2497620582580566,
  1.238168478012085,
  1.2492835521697998)}

- 가장 성능이 좋게 나온 값
 * RMSE: 1.6328
 * MAE: 1.2185
 * 성능간의 차이는 0.0035 정도로 나타남

## 알고리즘 성능 비교
| 알고리즘 이름 | 설명 | 장점 | 단점|
|---|---|---|---|
| **SVD**          | 잠재 요인 분석을 통해 사용자와 아이템의 특성을 벡터로 표현하여 평점을 예측 | - 예측 정확도 우수<br>- 잠재 요인을 통한 복잡한 패턴 학습 가능 | - 희소 데이터에서는 과적합 위험<br>- 연산 비용이 큼  |
| **KNNBasic**     | 유사한 사용자 또는 아이템을 기반으로 평점을 예측하는 메모리 기반 협업 필터링 | - 구현과 이해가 쉬움<br>- 추천 이유 설명 가능 (ex. 비슷한 유저/아이템 기반) | - 성능이 SVD보다 낮을 수 있음<br>- 대규모 데이터에서 느림 |
| **BaselineOnly** | 사용자와 아이템의 평균 편향만을 고려해 단순히 평점을 예측하는 베이스라인 모델 | - 빠르고 단순함<br>- 베이스라인 성능 파악에 유용  | - 복잡한 상호작용 반영 불가<br>- 추천 품질 낮음|



In [ ]:
from surprise import Dataset, SVD, KNNBasic, BaselineOnly
# SVD 알고리즘 교차 검증
print("=== SVD ===")
svd = SVD()  # 기본 파라미터로 SVD 알고리즘 사용
svd_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# KNNBasic 알고리즘 교차 검증
print("\n=== KNNBasic ===")
knn = KNNBasic()  # 기본 파라미터로 KNNBasic 알고리즘 사용
knn_results = cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# BaselineOnly 알고리즘 교차 검증
print("\n=== BaselineOnly ===")
base = BaselineOnly()  # 기본 파라미터로 BaselineOnly 알고리즘 사용
base_results = cross_validate(base, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# 결과 요약
print("\n=== Summary ===")
print(f"SVD mean RMSE: {sum(svd_results['test_rmse'])/5:.4f}, mean MAE: {sum(svd_results['test_mae'])/5:.4f}")
print(f"KNNBasic mean RMSE: {sum(knn_results['test_rmse'])/5:.4f}, mean MAE: {sum(knn_results['test_mae'])/5:.4f}")
print(f"BaselineOnly mean RMSE: {sum(base_results['test_rmse'])/5:.4f}, mean MAE: {sum(base_results['test_mae'])/5:.4f}")

=== SVD ===
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6372  1.6369  1.6312  1.6348  1.6399  1.6360  0.0029  
MAE (testset)     1.2228  1.2220  1.2189  1.2215  1.2228  1.2216  0.0014  
Fit time          11.37   11.81   11.90   11.87   11.99   11.79   0.21    
Test time         1.66    1.36    0.82    1.34    1.28    1.29    0.27    

=== KNNBasic ===
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.7974  1.7895  1.7950  

- SVD 성능 평균:
 * RMSE: 1.6360, MAE: 1.2216
- KNNBasic 성능 평균:
 * RMSE: 1.7913, MAE: 1.3279
- BaselineOnly 성능 평균:
 * RMSE: 1.6330, MAE: 1.2249

- RMSE는 **BaselineOnly**가 가장 성능이 좋게 나왔고, MAE는 **SVD**가 가장 성능이 좋게 나옴.
- 각각 std 추가 비교
 * SVD: 0.0029, 0.0014로 3 알고리즘 중 가장 편차가 낮음
 * BaselineOnly: 0.0039, 0.0031로 편차가 높게 나타남
- SVD의 경우 모델의 성능 편차가 가장 적음, 성능이 안정적으로 나올 가능성이 높음
- BaselineOnly의 경우 모델의 성능 편차가 가장 큼, 성능이 복불복일 가능성 높음

### 파라미터 튜닝 진행

- 사용 모델: SVD, BaselineOnly
 * SVD와 BaselineOnly의 교차 검증 결과
    * SVD의 경우 안정적으로 준수할 가능성이 높기에 채택.
    * BaselineOnly의 경우 편차는 높으나 성능이 KNN보다 준수하여 채택.

In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정.
# BaselineOnly does not have n_epochs or n_factors as parameters
param_grid = {}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=5)
gs.fit(data)
# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

In [8]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors':[50, 100, 200]}
                #에포크 학습 반복.        svd 잠재요인
# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
gs.fit(data)
# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.6558303421301865
{'n_epochs': 60, 'n_factors': 200}


In [9]:
best_p_svd = gs.best_params['rmse']
trainset_svd = data.build_full_trainset()
algo = SVD(n_epochs=best_p_svd['n_epochs'], n_factors=best_p_svd['n_factors'])
algo.fit(trainset_svd)

In [10]:
import pickle

# 피클파일로 저장
filename = 'svd_model.pkl'
pickle.dump(algo, open(filename, 'wb'))

print(f"Model saved to {filename}")

Model saved to svd_model.pkl


영화 종류는 총 99051개

In [11]:
movie = pd.read_csv('movie_name.csv')
movie

,movie_name
0,Nell
1,Ars amandi
2,Alarum
3,28 Years Later
4,We Live in Time
...,...
99046,Seethakaathi
99047,Vella Raja
99048,Samsaram Athu Minsaram
99049,Aravind SA: Madrasi Da


In [12]:
# user_id_to_check 에 user_id를 넣음
user_id_to_check = 'ur100419895'
rated_movies_by_user = ott[ott['user_id'] == user_id_to_check]['movie_name'].tolist()

# user가 보지 않은 영화 리스트 출력
unrated_movies = movie[~movie['movie_name'].isin(rated_movies_by_user)]

print(f"Number of movies not rated by user {user_id_to_check}: {len(unrated_movies)}")
display(unrated_movies)

Number of movies not rated by user ur100419895: 92668


,movie_name
0,Nell
1,Ars amandi
2,Alarum
3,28 Years Later
5,American Beach House
...,...
99046,Seethakaathi
99047,Vella Raja
99048,Samsaram Athu Minsaram
99049,Aravind SA: Madrasi Da


In [13]:
dictA = {'item_id':[],'est':[]}
for iid in list(unrated_movies['movie_name']):
    pred = algo.predict(user_id_to_check, iid)
    dictA['item_id'].append(iid)
    dictA['est'].append(pred.est)

result = pd.DataFrame(dictA)
result

,item_id,est
0,Nell,6.701229
1,Ars amandi,5.572957
2,Alarum,4.168811
3,28 Years Later,6.111686
4,American Beach House,5.065755
...,...,...
92663,Seethakaathi,6.047969
92664,Vella Raja,5.971646
92665,Samsaram Athu Minsaram,6.345043
92666,Aravind SA: Madrasi Da,5.799691


In [14]:
# est를 기준으로 정렬
result = result.sort_values('est', ascending=False)
result

,item_id,est
8698,Apocalypto,8.793334
19619,살아있는 지구 II,8.728958
26189,Ghost of Tsushima,8.656864
4658,마이클 조던: 더 라스트 댄스,8.651294
2728,Kingsman: Secret Agent,8.643784
...,...,...
16682,Spy Kids 3: Game Over,3.482255
28573,The View,3.442274
60750,앤 저스트 라이크 댓...,3.401504
23868,Winnie-the-Pooh: Blood and Honey,3.305959


### 최종
 - 고객의 ID를 입력할 경우 고객이 시청하지 않은 영화 데이터에 임의 평점을 예측함.
 - 예측한 평점을 기준으로 별점이 높은 순 10개를 노출하게함.

In [15]:
def movie_top_10(user_id):
    """
    주어진 사용자에 대해 예측 평점이 가장 높은 영화 10개를 추천합니다.

    Args:
        user_id (str): 추천을 받을 사용자의 ID.

    Returns:
        pandas.DataFrame: 추천 영화 목록과 예측 평점을 포함하는 DataFrame.
    """
    raw_ratings = data.raw_ratings
    all_items = set([r[1] for r in raw_ratings])
    rated_item_ids = set([r[1] for r in raw_ratings if r[0] == user_id])
    unrated_items = all_items - rated_item_ids

    dictA = {'item_id':[],'est':[]}
    for iid in list(unrated_items):
        pred = algo.predict(user_id, iid)
        dictA['item_id'].append(iid)
        dictA['est'].append(pred.est)

    result = pd.DataFrame(dictA)
    result = result.sort_values('est', ascending=False).head(10)

    # 영화 제목 정보 추가
    # df_movie DataFrame이 전역 변수로 정의되어 있다고 가정합니다.
    result = pd.merge(result, movie, left_on='item_id', right_on='movie_name')
    result = result.drop('movie_name', axis=1) # Remove the redundant movie_name column after merge
    result = result.rename(columns={'item_id': 'movie_name'}) # Rename item_id back to movie_name for clarity


    return result

In [30]:
movie_top_10('ur52780913')

,movie_name,est
0,쇼생크 탈출,8.921956
1,포레스트 검프,8.593931
2,스파이더맨: 노 웨이 홈,8.340650
3,다크 나이트,8.277779
4,The Matrix Reloaded,8.217366
5,기묘한 이야기,8.207797
6,칼 세이건의 코스모스,8.103592
7,퀸스 갬빗,8.068827
8,덱스터: 뉴 블러드,8.057907
9,David Attenborough: A Life on Our Planet,8.051672
